In [1]:
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import os
from datetime import datetime
import pdb;
from grocery_ml_tensorflow import GroceryML
from grocery_ml_core import GroceryMLCore
from hidden_layer_param_builder import HiddenLayerParamSetBuilder
import tensorflow as tf

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

print(os.getcwd())
# print("GPUs Available:", tf.config.list_physical_devices('GPU'))
#tf.debugging.set_log_device_placement(True)

# def run_all_experiments(df, model_param_sets, output_dir):
#     total = len(model_param_sets)
#     print(f"run_all_experiments() when: {datetime.now()}  output_dir: {output_dir}");
#     for index, params in enumerate(model_param_sets, 1):
#         print(f"Running Exp {index}/{total}...")
#         groceryML.run_experiment(df,  params["buildParams"], params["trainParams"], output_dir)


try:
    groceryML = GroceryML();
    groceryMLCore = GroceryMLCore();
    groceryML.build_training_df()
    if groceryML.training_df is None:
        raise();
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    # groceryML.training_df.to_csv(f"training_df-{ts}.csv");
except Exception as ex: 
    print(ex)
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    groceryML.training_df.to_csv(f"training_df-{ts}-exception.csv");


C:\Users\steve\source\repos\grocery-ml
_build_combined_df()
_build_sources()
creating target col: didBuy_target
insert_negative_samples()
compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
build_school_schedule_features()
build_holiday_features()
build_trip_interveral_feautres()
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done


In [3]:
import optuna 
def objective(trial):

    lr = trial.suggest_float("learning_rate", 0.0001, 0.0001)
    depth = trial.suggest_int("depth", 5, 5)
    units = trial.suggest_int("units", 5, 2048, step = 5)
    embedding_dim = trial.suggest_int("embedding_dim", 300, 300)
    epochs = trial.suggest_int("epochs", 50, 50)
    # output_activation = trial.suggest_categorical("output_activation", ["sigmoid", "linear"])
    output_activation = "sigmoid"
    metrics = ["AUC", "Precision", "Recall"]
    # metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]
    
    layers_cfg = []
    for _ in range(depth):
        layers_cfg.append({ "units": units, "activation": "relu" })

    build_params = {
        "embedding_dim": embedding_dim,
        "layers": layers_cfg,
        "output_activation": output_activation,
        "optimizer": "adam",
        "learning_rate": lr,
        "loss": "binary_crossentropy" if output_activation == "sigmoid" else "mse",
        "metrics": metrics
    }

    train_params = {
        "epochs": epochs,
        "batch_size": 32,
     
    }

    outputDir = f"f:/exp/keras/optuna/{trial.study.study_name}/trial-{trial.number}"
            
    groceryML.run_experiment(groceryML.training_df, build_params, train_params, outputDir)


    return groceryML.last_val_auc
############################################################################


sampler = optuna.samplers.TPESampler()

study_name = f"grocery_ml_tuning_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
study = optuna.create_study(
    study_name= study_name,
    sampler=sampler,
    direction="maximize",   # or minimize — see note below
    storage="sqlite:///optuna_grocery.db",
    load_if_exists=True
)


study.optimize(objective, n_trials=150)

[I 2026-01-15 14:01:35,644] A new study created in RDB with name: grocery_ml_tuning_20260115_140135
C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\site-packages\optuna\distributions.py:684: UserWarning: The distribution is specified by [5, 2048] and step=5, but the range is not divisible by `step`. It will be replaced with [5, 2045].
  warnings.warn(


Creating dir: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-0\e300_l470-470-470-470-470_ep50_
run_experiment()  exp_dir: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-0\e300_l470-470-470-470-470_ep50_  
run_experiment()  when: 2026-01-15 14:01:35.691391 params: {'epochs': 50, 'batch_size': 32}  
normalize_features()
train_model()
build_prediction_input() prediction_date=2026-01-15 14:02:00.616044
_build_combined_df()
_build_sources()
creating target col: didBuy_target
insert_negative_samples()


C:\Users\steve\source\repos\grocery-ml\grocery_ml_core.py:349: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  additional_rcpts_df["date"] = pd.to_datetime(additional_rcpts_df["date"])


compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
build_school_schedule_features()
build_holiday_features()
build_trip_interveral_feautres()
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
create_item_supply_level_feat()
normalize_features()
Running Model.Predict()
6/6 [==============================] - 0s 2ms/step
Exporting extra_dataframes:
grocery_ml_tensorflow.export_dataframes_to_excel()
Writing XLSX: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-0\e300_l470-470-470-470-470_ep50_\normalized_training_df-e300_l470-470-470-470-470_ep50_.xlsx
   XLSX Done: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-0\e300_l470-470-470-470-470_ep50_\normalized_training_df-e300_l470-470-470-470-470_ep50_.xlsx
Writing XLSX: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-0\e300_l470-470-470-470-470_ep50_\predictions-e300_l470-470-470-470-470_ep50_.xlsx
   XLSX Done: f:/exp/keras/optu

INFO:tensorflow:Assets written to: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-0\e300_l470-470-470-470-470_ep50_\model\assets


...

C:\Users\steve\source\repos\grocery-ml\grocery_ml_core.py:349: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  additional_rcpts_df["date"] = pd.to_datetime(additional_rcpts_df["date"])


creating target col: didBuy_target
insert_negative_samples()
compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
build_school_schedule_features()
build_holiday_features()
build_trip_interveral_feautres()
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
create_item_supply_level_feat()
normalize_features()
Running Model.Predict()
6/6 [==============================] - 0s 12ms/step
Exporting extra_dataframes:
grocery_ml_tensorflow.export_dataframes_to_excel()
Writing XLSX: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-149\e300_l1795-1795-1795-1795-1795_\normalized_training_df-e300_l1795-1795-1795-1795-1795_.xlsx
   XLSX Done: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-149\e300_l1795-1795-1795-1795-1795_\normalized_training_df-e300_l1795-1795-1795-1795-1795_.xlsx
Writing XLSX: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-149\e300_l1795-1795-1795-1795-1795_\predictions-

INFO:tensorflow:Assets written to: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-149\e300_l1795-1795-1795-1795-1795_\model\assets


INFO:tensorflow:Assets written to: f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-149\e300_l1795-1795-1795-1795-1795_\model\assets
[I 2026-01-16 00:26:23,108] Trial 149 finished with value: 0.8543822169303894 and parameters: {'learning_rate': 0.0001, 'depth': 5, 'units': 1795, 'embedding_dim': 300, 'epochs': 50}. Best is trial 70 with value: 0.9045388102531433.


[save_model] saving model weights (separate file)
[save_model] all artifacts saved successfully → f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-149\e300_l1795-1795-1795-1795-1795_\model
Saved experiment → f:/exp/keras/optuna/grocery_ml_tuning_20260115_140135/trial-149\e300_l1795-1795-1795-1795-1795_
